# Day 05: Risk Management Integration

## Week 24 - Capstone Project

### Learning Objectives
- Implement comprehensive risk metrics (VaR, CVaR, Expected Shortfall)
- Build position sizing frameworks (Kelly Criterion, Volatility Targeting)
- Create portfolio risk decomposition tools
- Integrate risk management into trading systems

### Why Risk Management is Critical
- **Capital Preservation**: Survive drawdowns to trade another day
- **Regulatory Compliance**: VaR limits, stress testing requirements
- **Position Sizing**: Optimal allocation based on risk
- **Drawdown Control**: Prevent catastrophic losses

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
from typing import Dict, List, Tuple
from dataclasses import dataclass

np.random.seed(42)
print("✅ Libraries loaded!")

## 1. Load Data

In [ ]:
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'GS']
START_DATE = '2019-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')

print("📥 Downloading data...")
data = yf.download(TICKERS, start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)
prices = data['Close'].dropna()
returns = prices.pct_change().dropna()

print(f"✅ Data loaded: {returns.shape[0]} days, {returns.shape[1]} tickers")

## 2. Value at Risk (VaR) Implementation

In [ ]:
@dataclass
class RiskMetrics:
    """Container for risk metrics."""
    var_95: float
    var_99: float
    cvar_95: float
    cvar_99: float
    volatility: float
    max_drawdown: float
    
    def summary(self) -> str:
        return f"""
Risk Metrics:
  VaR (95%):   {self.var_95*100:>8.2f}%
  VaR (99%):   {self.var_99*100:>8.2f}%
  CVaR (95%):  {self.cvar_95*100:>8.2f}%
  CVaR (99%):  {self.cvar_99*100:>8.2f}%
  Volatility:  {self.volatility*100:>8.2f}%
  Max DD:      {self.max_drawdown*100:>8.2f}%
"""


class RiskManager:
    """Comprehensive risk management system."""
    
    def __init__(self, confidence_levels: List[float] = [0.95, 0.99]):
        self.confidence_levels = confidence_levels
    
    def calculate_var_historical(self, returns: pd.Series, confidence: float = 0.95) -> float:
        """Historical VaR (percentile method)."""
        return np.percentile(returns, (1 - confidence) * 100)
    
    def calculate_var_parametric(self, returns: pd.Series, confidence: float = 0.95) -> float:
        """Parametric VaR (assuming normal distribution)."""
        mu = returns.mean()
        sigma = returns.std()
        z_score = stats.norm.ppf(1 - confidence)
        return mu + z_score * sigma
    
    def calculate_var_monte_carlo(self, returns: pd.Series, confidence: float = 0.95, 
                                   n_simulations: int = 10000) -> float:
        """Monte Carlo VaR."""
        mu = returns.mean()
        sigma = returns.std()
        simulated = np.random.normal(mu, sigma, n_simulations)
        return np.percentile(simulated, (1 - confidence) * 100)
    
    def calculate_cvar(self, returns: pd.Series, confidence: float = 0.95) -> float:
        """Conditional VaR (Expected Shortfall)."""
        var = self.calculate_var_historical(returns, confidence)
        return returns[returns <= var].mean()
    
    def calculate_metrics(self, returns: pd.Series) -> RiskMetrics:
        """Calculate all risk metrics."""
        # VaR and CVaR
        var_95 = self.calculate_var_historical(returns, 0.95)
        var_99 = self.calculate_var_historical(returns, 0.99)
        cvar_95 = self.calculate_cvar(returns, 0.95)
        cvar_99 = self.calculate_cvar(returns, 0.99)
        
        # Volatility
        volatility = returns.std() * np.sqrt(252)
        
        # Max Drawdown
        cumulative = (1 + returns).cumprod()
        cummax = cumulative.cummax()
        drawdown = (cumulative - cummax) / cummax
        max_drawdown = abs(drawdown.min())
        
        return RiskMetrics(
            var_95=abs(var_95),
            var_99=abs(var_99),
            cvar_95=abs(cvar_95),
            cvar_99=abs(cvar_99),
            volatility=volatility,
            max_drawdown=max_drawdown
        )

# Calculate risk metrics for each ticker
risk_manager = RiskManager()

print("="*60)
print("RISK METRICS BY TICKER")
print("="*60)

for ticker in TICKERS:
    metrics = risk_manager.calculate_metrics(returns[ticker])
    print(f"\n{ticker}:")
    print(metrics.summary())

## 3. Position Sizing Frameworks

In [ ]:
class PositionSizer:
    """Position sizing frameworks."""
    
    def __init__(self, capital: float, risk_per_trade: float = 0.02):
        """
        Args:
            capital: Total portfolio capital
            risk_per_trade: Maximum risk per trade (default 2%)
        """
        self.capital = capital
        self.risk_per_trade = risk_per_trade
    
    def fixed_fraction(self, price: float, fraction: float = 0.10) -> int:
        """Simple fixed fraction position sizing."""
        position_value = self.capital * fraction
        return int(position_value / price)
    
    def volatility_targeting(self, price: float, volatility: float, 
                              target_vol: float = 0.15) -> int:
        """
        Size position to target portfolio volatility.
        
        Args:
            price: Current price
            volatility: Annualized volatility of the asset
            target_vol: Target portfolio volatility (default 15%)
        """
        leverage = target_vol / volatility
        position_value = self.capital * min(leverage, 1.0)  # Cap at 100%
        return int(position_value / price)
    
    def kelly_criterion(self, price: float, win_rate: float, 
                        avg_win: float, avg_loss: float, 
                        kelly_fraction: float = 0.5) -> int:
        """
        Kelly Criterion for optimal position sizing.
        
        f* = (bp - q) / b
        where:
            b = odds (avg_win / avg_loss)
            p = win probability
            q = loss probability (1 - p)
        
        Args:
            kelly_fraction: Fraction of Kelly to use (default 0.5 = half-Kelly)
        """
        b = avg_win / avg_loss if avg_loss > 0 else 1
        p = win_rate
        q = 1 - p
        
        kelly = (b * p - q) / b
        kelly = max(0, kelly)  # No shorting
        kelly = min(kelly * kelly_fraction, 1.0)  # Apply fraction, cap at 100%
        
        position_value = self.capital * kelly
        return int(position_value / price)
    
    def risk_based(self, price: float, stop_loss_pct: float) -> int:
        """
        Size based on maximum acceptable loss.
        
        If stop loss is 5%, and max risk is 2% of capital,
        position size = (capital * 0.02) / (price * 0.05)
        """
        max_loss_dollars = self.capital * self.risk_per_trade
        loss_per_share = price * stop_loss_pct
        return int(max_loss_dollars / loss_per_share)

# Example position sizing
sizer = PositionSizer(capital=100000, risk_per_trade=0.02)
example_price = prices['AAPL'].iloc[-1]
example_vol = returns['AAPL'].std() * np.sqrt(252)

print("="*60)
print("POSITION SIZING COMPARISON (AAPL)")
print("="*60)
print(f"Current Price: ${example_price:.2f}")
print(f"Annualized Vol: {example_vol*100:.1f}%")
print(f"\nMethod                    Shares     Position Value")
print("-"*60)

methods = [
    ('Fixed Fraction (10%)', sizer.fixed_fraction(example_price, 0.10)),
    ('Volatility Target (15%)', sizer.volatility_targeting(example_price, example_vol, 0.15)),
    ('Half-Kelly (55% win)', sizer.kelly_criterion(example_price, 0.55, 0.02, 0.015, 0.5)),
    ('Risk-Based (5% SL)', sizer.risk_based(example_price, 0.05))
]

for name, shares in methods:
    value = shares * example_price
    print(f"{name:<25} {shares:>6}     ${value:>10,.2f}")

## 4. Portfolio Risk Decomposition

In [ ]:
class PortfolioRiskAnalyzer:
    """Portfolio risk decomposition and analysis."""
    
    def __init__(self, returns: pd.DataFrame):
        self.returns = returns
        self.cov_matrix = returns.cov() * 252  # Annualized
        self.corr_matrix = returns.corr()
    
    def portfolio_volatility(self, weights: np.ndarray) -> float:
        """Calculate portfolio volatility."""
        return np.sqrt(weights @ self.cov_matrix @ weights)
    
    def marginal_contribution_to_risk(self, weights: np.ndarray) -> np.ndarray:
        """Calculate marginal contribution to risk (MCR)."""
        port_vol = self.portfolio_volatility(weights)
        mcr = (self.cov_matrix @ weights) / port_vol
        return mcr
    
    def component_contribution_to_risk(self, weights: np.ndarray) -> np.ndarray:
        """Calculate component contribution to risk (CCR)."""
        mcr = self.marginal_contribution_to_risk(weights)
        return weights * mcr
    
    def percent_contribution_to_risk(self, weights: np.ndarray) -> np.ndarray:
        """Calculate percent contribution to risk."""
        ccr = self.component_contribution_to_risk(weights)
        port_vol = self.portfolio_volatility(weights)
        return ccr / port_vol
    
    def risk_parity_weights(self) -> np.ndarray:
        """Calculate risk parity weights."""
        n = len(self.returns.columns)
        
        def objective(weights):
            pcr = self.percent_contribution_to_risk(weights)
            return np.sum((pcr - 1/n) ** 2)
        
        constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
        bounds = [(0.01, 0.5) for _ in range(n)]
        
        result = minimize(
            objective,
            x0=np.ones(n) / n,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints
        )
        
        return result.x

# Analyze portfolio risk
analyzer = PortfolioRiskAnalyzer(returns)

# Equal weight portfolio
equal_weights = np.ones(len(TICKERS)) / len(TICKERS)

# Risk parity weights
rp_weights = analyzer.risk_parity_weights()

print("="*70)
print("PORTFOLIO RISK DECOMPOSITION")
print("="*70)

print("\n📊 EQUAL WEIGHT PORTFOLIO")
print("-"*50)
pcr = analyzer.percent_contribution_to_risk(equal_weights)
port_vol = analyzer.portfolio_volatility(equal_weights)

print(f"Portfolio Volatility: {port_vol*100:.2f}%")
print(f"\n{'Ticker':<10} {'Weight':<10} {'Risk Contrib':<15}")
print("-"*35)
for i, ticker in enumerate(TICKERS):
    print(f"{ticker:<10} {equal_weights[i]*100:>7.1f}%    {pcr[i]*100:>10.1f}%")

print("\n📊 RISK PARITY PORTFOLIO")
print("-"*50)
pcr_rp = analyzer.percent_contribution_to_risk(rp_weights)
port_vol_rp = analyzer.portfolio_volatility(rp_weights)

print(f"Portfolio Volatility: {port_vol_rp*100:.2f}%")
print(f"\n{'Ticker':<10} {'Weight':<10} {'Risk Contrib':<15}")
print("-"*35)
for i, ticker in enumerate(TICKERS):
    print(f"{ticker:<10} {rp_weights[i]*100:>7.1f}%    {pcr_rp[i]*100:>10.1f}%")

## 5. Trading Signals with Risk Management

In [ ]:
print("\n" + "="*70)
print("🎯 TODAY'S RISK-ADJUSTED TRADING SIGNALS")
print("="*70)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"Portfolio Capital: $100,000")
print(f"Risk Per Trade: 2%\n")

risk_manager = RiskManager()
sizer = PositionSizer(capital=100000, risk_per_trade=0.02)

for ticker in TICKERS:
    price = prices[ticker].iloc[-1]
    ret = returns[ticker]
    metrics = risk_manager.calculate_metrics(ret)
    
    # Calculate momentum
    momentum_20d = (prices[ticker].iloc[-1] / prices[ticker].iloc[-20] - 1)
    
    # Calculate volatility-adjusted position size
    vol = ret.std() * np.sqrt(252)
    shares = sizer.volatility_targeting(price, vol, target_vol=0.15)
    position_value = shares * price
    
    # Signal based on momentum and risk
    if momentum_20d > 0.05 and metrics.volatility < 0.35:
        signal = "🟢 BUY"
        action = "CALL options or shares"
    elif momentum_20d < -0.05 or metrics.volatility > 0.40:
        signal = "🔴 SELL/AVOID"
        action = "PUT options or reduce"
    else:
        signal = "⚪ HOLD"
        action = "Maintain position"
    
    print(f"{'='*30} {ticker} {'='*30}")
    print(f"   Price:        ${price:.2f}")
    print(f"   20d Momentum: {momentum_20d*100:+.2f}%")
    print(f"   Volatility:   {vol*100:.1f}%")
    print(f"   VaR (95%):    {metrics.var_95*100:.2f}%")
    print(f"   Signal:       {signal}")
    print(f"   Action:       {action}")
    print(f"   Position:     {shares} shares (${position_value:,.0f})")
    print(f"   Stop Loss:    ${price * 0.95:.2f} (-5%)")
    print(f"   Take Profit:  ${price * 1.10:.2f} (+10%)")
    print()

## 6. Key Takeaways

### Risk Management Framework:
1. **VaR & CVaR**: Quantify tail risk
2. **Position Sizing**: Volatility targeting, Kelly Criterion
3. **Risk Decomposition**: Understand where risk comes from
4. **Stop Loss/Take Profit**: Mechanical exit rules

### Tomorrow: Deployment & Monitoring
- Model deployment patterns
- Real-time monitoring
- Alert systems